In [1]:
import time
import json
import regex as re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Константи пошуку

PROM = 'https://prom.ua/ua/'
FIND = 'світлодіод 5мм'

XPATH_SEARCH = '//input[@type="search"]'
XPATH_SUBMIT = '//button[@type="submit"]'
XPATH_PRODUCT_BLOCK = '//div[@data-qaid = "product_gallery"]/div[@data-qaid = "product_block"]//a[@data-qaid = "product_link"]'
XPATH_PRODUCT_SET_PRESENT = '//button[@data-qaid = "presence_available"]'
XPATH_PAGE_BUTTON = '//a[@data-qaid = "pages"]'

REGEX_COLOR = 'інфрачервоний|червоний|жовтий|зелений|синій|ультрафіолетовий|білий|rgb|набір'
REGEX_AMOUNT = ' (\d{1,})(шт|х|x)(. |, | )'
REGEX_CURRENCY = '"currency":"(\w{1,})"'
REGEX_DISCOUNT = '"discount":(\d{1,})'
REGEX_PRICE = '"price":(\d{1,})'

In [3]:
# Парсинг

driver = webdriver.Chrome(r'C:\Users\user\chromedriver-win64\chromedriver.exe')

yeild_title = []
yeild_url = []
yeild_amount = []
yeild_currency = []
yeild_discount = []
yeild_price = []
yeild_ppu = [] # Price per unit
yeild_color = []

driver.get(PROM) # Відкрити пром

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.XPATH, XPATH_SEARCH)))   # Затримка для загрузки

search = driver.find_element(By.XPATH, XPATH_SEARCH)   # Знаходить пошуку
search.send_keys(FIND)

button = driver.find_element(By.XPATH, XPATH_SUBMIT)   # Нажимає на кнопку пошуку
button.click()

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.XPATH, XPATH_PRODUCT_SET_PRESENT)))   # Затримка для загрузки

button = driver.find_element(By.XPATH, XPATH_PRODUCT_SET_PRESENT)   # Прибирає неприсутні офери
button.click()

jump = True
idx = 0
PAGE_NUMBER = 1

while jump :   # Поки є сторінки, парсимо
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.XPATH, XPATH_PRODUCT_BLOCK)))   # Затримка для загрузки
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")   # Пром не грузить все якщо не прокручено вниз
    time.sleep(3) # Пром повільно завантажується
    
    for product in driver.find_elements(By.XPATH, XPATH_PRODUCT_BLOCK) :
        yeild_title.append(product.get_attribute('title'))                                    # Title
        
        yeild_url.append(product.get_attribute('href'))                                       # Href
        
        yeild_amount.append(re.findall(REGEX_AMOUNT, yeild_title[idx].lower()))               # Amount
        if len(yeild_amount[idx]) == 0 : yeild_amount[idx] = [[1]]
        yeild_amount[idx] = int(yeild_amount[idx][0][0])
        
        data_tg_clicked = product.get_attribute('data-tg-clicked')
        
        yeild_currency.append(re.findall(REGEX_CURRENCY, data_tg_clicked)[0])                 # Currency
        
        yeild_discount.append(int(re.findall(REGEX_DISCOUNT, data_tg_clicked)[0]))            # Discount
        
        yeild_price.append(int(re.findall(REGEX_PRICE, data_tg_clicked)[0]))                  # Price
        yeild_price[idx] -= yeild_discount[idx]
        
        yeild_color.append(re.findall(REGEX_COLOR, yeild_title[idx].lower()))                 # Color
        if len(yeild_color[idx]) == 0 : yeild_color[idx] = ['інше']
        yeild_color[idx] = yeild_color[idx][0]
        
        yeild_ppu.append(yeild_price[idx] / yeild_amount[idx])                                # Ppu
        
        if idx % 100 == 0 :   # Приклади
            print(yeild_title[idx])
            print(yeild_url[idx])
            print("Кількість :", yeild_amount[idx])
            print("Колір :", yeild_color[idx])
            print("Ціна :", yeild_price[idx], yeild_currency[idx])
            print("Має знижку :", yeild_discount[idx], yeild_currency[idx])
            print("Ціна за 1 шт :", yeild_ppu[idx], yeild_currency[idx])
            print()
        
        idx += 1
    
    #break   # Якщо треба парсити лише першу сторінку
    
    jump = False
    PAGE_NUMBER += 1
    for button in driver.find_elements(By.XPATH, XPATH_PAGE_BUTTON) :
        if button.text == str(PAGE_NUMBER) :
            jump = True
            driver.execute_script("window.scrollTo(0, 0);")   # Кнопка не може бути нажата якщо прокрутити вниз
            time.sleep(1)
            button.click()
            break
    # Перевірка наявності дати, має бути 29 за одну сторінку
    print("Found", idx, "offers")
    print()
driver.quit()

1000x LED світлодіод 5 мм 3-3.2 В 20 мА, синій GJH
https://prom.ua/ua/p2288387900-1000x-led-svetodiod.html?token=v2%3Azuvx0PnyJyYBUsN8xagv7zMSCRgNrKQvF-iUrDEZOsPqkcLRM6M1C8AmuXmkxw_TBhUn-HsYQ5rwn3jxrB6r9rOWzOSJaiuDHv_LpEUwvzaioVH7zqPZROhbQT8af3s_&campaign_id=3979190&product_id=2288387900&source=prom%3Asearch%3Aserp&locale=uk&category_ids=53008&from_spa=true
Кількість : 1
Колір : синій
Ціна : 657 UAH
Має знижку : 328 UAH
Ціна за 1 шт : 657.0 UAH

Found 29 offers

Found 58 offers

Found 87 offers

1000x LED світлодіод 5мм 1.8-2В 20мА, зелений
https://prom.ua/ua/p1030123669-1000x-led-svetodiod.html?token=v2%3AFiXDMwNMoznkwkAXwuEBxx46NmoITuSMCgK9AiQLvil-N_JSObgoXBucYJk3inxepP7wnafrUwmxItKRLUQF8OWrOMCU44ROYOyuVXbbyHJhYz_yWlR27s_yWkAjWwMJ&campaign_id=3513734&product_id=1030123669&source=prom%3Asearch%3Aserp&locale=uk&category_ids=53008&from_spa=true
Кількість : 1
Колір : зелений
Ціна : 390 UAH
Має знижку : 62 UAH
Ціна за 1 шт : 390.0 UAH

Found 116 offers

Found 145 offers

Found 174 offers


Found 1566 offers

Found 1595 offers

LED COB світлодіод матриця 13,5 х13, 5мм 15W 45-49V 300mA для трекового світильника Тепло-білий
https://prom.ua/ua/p2058249838-led-cob-svetodiod.html?token=v2%3A-cJ_s7PTnte41T7Ol0PpqebOCJS6LxvHMExV_a7Gu_g3NSgnbslhbdyH2Iw4eolPuUzYpfJ43YzRATkgQiCrtgRGkJco0EeEzIG6IyJNaJnXoWnZ2rujZSGKKLoPgLq9&campaign_id=3779101&product_id=2058249838&source=prom%3Asearch%3Aserp&locale=uk&category_ids=53008&from_spa=true
Кількість : 1
Колір : білий
Ціна : 58 UAH
Має знижку : 0 UAH
Ціна за 1 шт : 58.0 UAH

Found 1624 offers

Found 1653 offers

Found 1682 offers

Found 1691 offers



In [4]:
filename = "JSON.json" # Зберігаємо дату
data = [
    {
        "title": yeild_title[i],
        "url" : yeild_url[i],
        "amount" : yeild_amount[i],
        "currency" : yeild_currency[i],
        "discount" : yeild_discount[i],
        "price" : yeild_price[i],
        "ppu" : yeild_ppu[i],
        "color" : yeild_color[i]
        
    }
    for i in range(0, len(yeild_title))
]
with open(filename, "w", encoding='utf-8') as f:
    json.dump(data, f, indent=2, ensure_ascii=False)